In [15]:
import os
import re
import json
from core.config.config_loader import ConfigLoader
from core.pilot.harmony.component.components import get_harmony_component
from core.pilot.harmony.resource import load_harmony_resource
from core.pilot.harmony.utils import get_component_related_types
from core.pilot.schema import BreakdownLayout, BreakdownLayoutTranslation, ChooseComponent, Files, \
    BreakdownComponentTranslation
from core.prompt.prompt_loader import PromptLoader
from template.harmony_empty_ability import HarmonyEmptyAbilityV5ProjectTemplate
from core.llms.oai_client import OpenAIClient
from core.logger.runtime import get_logger

os.chdir("D:/Codes/Python/harmony-pilot")

logger = get_logger(name="harmony-pilot test")


def extract_code_blocks(markdown_text):
    """从 Markdown 文本中提取代码块"""
    code_block_pattern = re.compile(r'```.*?\n(.*?)\n```', re.DOTALL)
    code_block_match = code_block_pattern.search(markdown_text)
    if code_block_match:
        return code_block_match.group(1)
    return markdown_text

In [2]:
# ======================== 从文件中加载配置 ========================
ConfigLoader.from_file(r"D:\Codes\Python\harmony-pilot\config.yaml")
harmony_template = HarmonyEmptyAbilityV5ProjectTemplate()
openai_default_config = ConfigLoader.get_config().llm_config.get("deepseek").model_dump()
resource = load_harmony_resource(r"D:\Codes\ArkTS\dashbook\entry\src\main\resources")
client = OpenAIClient(openai_default_config)

In [39]:
# ======================== 配置待转译页面 ========================
layout_files = {
    "app/res/xml/app_preferences.xml": {
        "name": "app/res/xml/app_preferences.xml",
        "content": "<androidx.preference.PreferenceScreen\nxmlns:android=\"http://schemas.android.com/apk/res/android\">\n\n<androidx.preference.Preference\nandroid:key=\"tutorial_display_key\"\nandroid:summary=\"@string/show_tutorial_settings_summary\"\nandroid:title=\"@string/show_tutorial_settings_title\" />\n\n<androidx.preference.SwitchPreferenceCompat\nandroid:defaultValue=\"true\"\nandroid:key=\"pref_new_book_notification\"\nandroid:summary=\"@string/settings_new_book_notifications_summary\"\nandroid:title=\"@string/settings_new_book_notifications\"\n/>\n</androidx.preference.PreferenceScreen>",
        "java": "",
        "contains": [],
        "resources": {
            "@string/show_tutorial_settings_summary": "resources/values/strings.xml",
            "@string/show_tutorial_settings_title": "resources/values/strings.xml",
            "@string/settings_new_book_notifications_summary": "resources/values/strings.xml",
            "@string/settings_new_book_notifications": "resources/values/strings.xml"

        }
    }
}

In [40]:
# ======================== 拆分转译组件 ========================
# ======================== Developer Agent ========================
current_developer_task_index = 0
android_layout_xml_name = list(layout_files.keys())[current_developer_task_index]

layout_translation = BreakdownLayoutTranslation(
    tasks=[{
        "description": f"将{android_layout_xml_name}转译为对应的Harmony页面",
        "done": False,
        "android": android_layout_xml_name,
        "harmony": "ets/pages/Index.ets"
    }]
)

developer_system_prompt = PromptLoader.get_prompt("developer/system.prompt")
developer_translate_layout_plan_prompt = PromptLoader.get_prompt(
    "developer/breakdown_layout.prompt",
    tasks=layout_translation.tasks,
    current_task=layout_translation.tasks[current_developer_task_index],
    android_layout=layout_files[layout_translation.tasks[current_developer_task_index].android],
    harmony_components=get_harmony_component(),
    is_component_content=False,
    is_type_content=True,
    project_resources=resource
)
developer_response = client.create(messages=[
    {"content": developer_system_prompt, "role": "system"},
    {"content": developer_translate_layout_plan_prompt, "role": "user"}
], model_schema=BreakdownLayout, temperature=0.0)
print("============================== 转译计划 ==============================")
print(developer_response.choices[0].message.content)

============================== 转译计划 ==============================
```json
{
    "tasks": [
        {
            "description": "创建ets/pages/SettingsPage.ets文件，并定义页面布局。",
            "done": false,
            "component": ""
        },
        {
            "description": "在ets/pages/SettingsPage.ets文件中添加Switch组件，用于显示和控制新书通知的开关状态。该组件的默认值为true，标题为'新书通知'，摘要为'接收新书的通知'。",
            "done": false,
            "component": "<androidx.preference.SwitchPreferenceCompat\nandroid:defaultValue=\"true\"\nandroid:key=\"pref_new_book_notification\"\nandroid:summary=\"@string/settings_new_book_notifications_summary\"\nandroid:title=\"@string/settings_new_book_notifications\"\n/>"
        },
        {
            "description": "在ets/pages/SettingsPage.ets文件中添加Preference组件，用于显示教程设置的标题和摘要。标题为'显示教程'，摘要为'在启动时显示教程'。",
            "done": false,
            "component": "<androidx.preference.Preference\nandroid:key=\"tutorial_display_key\"\nandroid:summary=\"@string/show_tutorial_settings_summary\"\nan

In [43]:
# ======================== 根据转译任务进行转译 ========================
layout_translation = BreakdownComponentTranslation.model_validate(
    json.loads(extract_code_blocks(developer_response.choices[0].message.content)))
current_code_monkey_task_index = 1
components = [
    "Text",
    # "Scroll",
    # "Divider",
    # "Row",
    # "Column",
    "Button",
    "Toggle"
    # "Image",
    # "Blank",
    # "ToolBar"
]

# 查询组件的文档
related_component = {}
for component in components:
    related_component.update(get_harmony_component(component))

# 查询组件相关类型定义
harmony_types = get_component_related_types(list(related_component.keys()))

# 整个安卓页面的内容
android_layout = {
    "name": android_layout_xml_name,
    "content": layout_files[android_layout_xml_name]["content"]
}
harmony_layout = {
    "name": "ets/pages/Index.ets",
    "content": ""
}

In [44]:
# ======================== Code Monkey Agent ========================
code_monkey_system_prompt = PromptLoader.get_prompt("code_monkey/system.prompt")
code_monkey_translate_layout_prompt = PromptLoader.get_prompt(
    "code_monkey/translate_layout.prompt",
    tasks=layout_translation.tasks,
    current_task=layout_translation.tasks[current_code_monkey_task_index],
    android_layout=android_layout,
    harmony_layout=harmony_layout,
    harmony_components=related_component,
    is_component_content=True,
    harmony_types=harmony_types,
    is_type_content=True,
    project_resources=resource
)

messages = [
    {"content": code_monkey_system_prompt, "role": "system"},
    {"content": code_monkey_translate_layout_prompt, "role": "user"}
]
# while current_code_monkey_task_index < len(layout_translation.tasks):
print("============================== 当前任务 ==============================")
print(layout_translation.tasks[current_code_monkey_task_index].description)
code_monkey_response = client.create(messages=messages, temperature=0.0)
print("============================== Code Monkey 回复 ==============================")
print(code_monkey_response.choices[0].message.content)
harmony_code = extract_code_blocks(code_monkey_response.choices[0].message.content)
code_monkey_translate_layout_prompt = PromptLoader.get_prompt(
    "code_monkey/translate_layout.prompt",
    tasks=layout_translation.tasks,
    current_task=layout_translation.tasks[current_code_monkey_task_index],
    android_layout=android_layout,
    harmony_layout=harmony_layout,
    harmony_components=related_component,
    is_component_content=True,
    harmony_types=harmony_types,
    is_type_content=True,
    project_resources=resource
)
messages = [
    {"content": code_monkey_system_prompt, "role": "system"},
    {"content": code_monkey_translate_layout_prompt, "role": "user"}
]
print("============================== Harmony 代码 ==============================")
print(harmony_code)
harmony_layout["content"] += harmony_code
current_code_monkey_task_index += 1
# input()


============================== 当前任务 ==============================
在ets/pages/SettingsPage.ets文件中添加Switch组件，用于显示和控制新书通知的开关状态。该组件的默认值为true，标题为'新书通知'，摘要为'接收新书的通知'。
============================== Code Monkey 回复 ==============================
根据提供的安卓布局代码和转译规则，以下是转译后的鸿蒙ArkUI代码：

```ets
// ets/pages/SettingsPage.ets

@Entry
@Component
struct SettingsPage {
  @State isNotificationEnabled: boolean = true;

  build() {
    Column() {
      Text($r("app.string.settings_new_book_notifications"))
        .fontSize(20)
        .fontWeight(FontWeight.Bold)
        .margin({ bottom: 8 })

      Text($r("app.string.settings_new_book_notifications_summary"))
        .fontSize(16)
        .fontColor(0x666666)
        .margin({ bottom: 16 })

      Toggle({ type: ToggleType.Switch, isOn: this.isNotificationEnabled })
        .selectedColor('#007DFF')
        .onChange((isOn: boolean) => {
          this.isNotificationEnabled = isOn;
        })
    }
    .padding(16)
    .width('100%')
  }
}
```

### 解释：


In [45]:
print("============================== 当前任务 ==============================")
print(layout_translation.tasks[current_code_monkey_task_index].description)
print("============================== Code Monkey Prompt ==============================")
print(messages[-1]["content"])
code_monkey_response = client.create(messages=messages, temperature=0.0)
print("============================== Code Monkey 回复 ==============================")
print(code_monkey_response.choices[0].message.content)
harmony_code = extract_code_blocks(code_monkey_response.choices[0].message.content)
code_monkey_translate_layout_prompt = PromptLoader.get_prompt(
    "code_monkey/translate_layout.prompt",
    tasks=layout_translation.tasks,
    current_task=layout_translation.tasks[current_code_monkey_task_index],
    android_layout=android_layout,
    harmony_layout=harmony_layout,
    harmony_components=related_component,
    is_component_content=True,
    harmony_types=harmony_types,
    is_type_content=True,
    project_resources=resource
)
messages = [
    {"content": code_monkey_system_prompt, "role": "system"},
    {"content": code_monkey_translate_layout_prompt, "role": "user"}
]
print("============================== Harmony 代码 ==============================")
print(harmony_code)
harmony_layout["content"] += harmony_code
current_code_monkey_task_index += 1

============================== 当前任务 ==============================
在ets/pages/SettingsPage.ets文件中添加Preference组件，用于显示教程设置的标题和摘要。标题为'显示教程'，摘要为'在启动时显示教程'。
============================== Code Monkey Prompt ==============================
你在一家软件公司工作，你被指派去为将安卓布局文件转译为鸿蒙ArkUI代码。

技术主管已经将代码转译分为以下步骤：
1. 创建ets/pages/SettingsPage.ets文件，并定义页面布局。\n
2. 在ets/pages/SettingsPage.ets文件中添加Switch组件，用于显示和控制新书通知的开关状态。该组件的默认值为true，标题为'新书通知'，摘要为'接收新书的通知'。\n
3. 在ets/pages/SettingsPage.ets文件中添加Preference组件，用于显示教程设置的标题和摘要。标题为'显示教程'，摘要为'在启动时显示教程'。\n

你当前需要处理的任务是：
在ets/pages/SettingsPage.ets文件中添加Switch组件，用于显示和控制新书通知的开关状态。该组件的默认值为true，标题为'新书通知'，摘要为'接收新书的通知'。

以下是可能需要用到的ArkUI类型定义：
以下是关于鸿蒙ArkUI API 12的类型定义基本信息。

## ControlSize
该枚举类定义了按钮的尺寸类型
枚举值：['SMALL', 'NORMAL']
枚举值介绍：
* SMALL：小尺寸按钮
* NORMAL：正常尺寸按钮
## CustomBuilder
组件属性方法参数可使用CustomBuilder类型来自定义UI描述。
参数：{'CustomBuilder': {"type":"() => any | void","required":false,"description":"生成用户自定义组件，在使用时结合@Builder使用。","enum":null,"enumDescriptions":null,"default":null}}
## Tex

In [28]:
print("============================== 当前任务 ==============================")
print(layout_translation.tasks[current_code_monkey_task_index].description)
print("============================== Code Monkey Prompt ==============================")
print(messages[-1]["content"])
code_monkey_response = client.create(messages=messages, temperature=0.0)
print("============================== Code Monkey 回复 ==============================")
print(code_monkey_response.choices[0].message.content)
harmony_code = extract_code_blocks(code_monkey_response.choices[0].message.content)
code_monkey_translate_layout_prompt = PromptLoader.get_prompt(
    "code_monkey/translate_layout.prompt",
    tasks=layout_translation.tasks,
    current_task=layout_translation.tasks[current_code_monkey_task_index],
    android_layout=android_layout,
    harmony_layout=harmony_layout,
    harmony_components=related_component,
    is_component_content=True,
    harmony_types=harmony_types,
    is_type_content=True,
    project_resources=resource
)
messages = [
    {"content": code_monkey_system_prompt, "role": "system"},
    {"content": code_monkey_translate_layout_prompt, "role": "user"}
]
print("============================== Harmony 代码 ==============================")
print(harmony_code)
harmony_layout["content"] += harmony_code
current_code_monkey_task_index += 1

============================== 当前任务 ==============================
在ets/pages/AboutPage.ets文件中的LinearLayout组件内添加第一个CardView组件，并设置其布局和样式与安卓布局文件中的第一个CardView一致。
============================== Code Monkey Prompt ==============================
你在一家软件公司工作，你被指派去为将安卓布局文件转译为鸿蒙ArkUI代码。

技术主管已经将代码转译分为以下步骤：
1. 创建ets/pages/AboutPage.ets文件，并定义页面布局。\n
2. 在ets/pages/AboutPage.ets文件中添加ScrollView组件，并设置其布局和样式与安卓布局文件中的ScrollView一致。\n
3. 在ets/pages/AboutPage.ets文件中的ScrollView组件内添加LinearLayout组件，并设置其布局和样式与安卓布局文件中的LinearLayout一致。\n
4. 在ets/pages/AboutPage.ets文件中的LinearLayout组件内添加第一个CardView组件，并设置其布局和样式与安卓布局文件中的第一个CardView一致。\n
5. 在ets/pages/AboutPage.ets文件中的第一个CardView组件内添加TextView组件，并设置其布局和样式与安卓布局文件中的TextView一致。\n
6. 在ets/pages/AboutPage.ets文件中的LinearLayout组件内添加第二个CardView组件，并设置其布局和样式与安卓布局文件中的第二个CardView一致。\n
7. 在ets/pages/AboutPage.ets文件中的第二个CardView组件内添加ImageView组件，并设置其布局和样式与安卓布局文件中的ImageView一致。\n
8. 在ets/pages/AboutPage.ets文件中的LinearLayout组件内添加第三个CardView组件，并设置其布局和样式与安卓布局文件中的第三个CardView一致。\n
9. 在ets/pag